In [2]:
import glob
from PIL import Image
import subprocess
import os
import json

In [3]:
def make_folder(folder_name):
    """ 
    Function to make a new folder (if not there)
    """
    "Creating folder to save ..."
    folder_path = "./"+folder_name
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print("\nFolder named "+folder_name+" created! \n")
    else:
        print('\nFolder '+folder_name+" already exists! \nreplacing existing frames (if any)\n")

In [4]:
def video_info(mp4_file_path,basename):
    """
    Function to get Video file info
    """
    command_video = [
    'ffprobe',
    '-v', 'error',          # Hide unnecessary warnings
    '-select_streams', 'v:0', # Select the first video stream
    '-show_entries', 'stream=r_frame_rate,codec_name,pix_fmt', # Show the real frame rate
    '-of', 'json',           # Output as JSON
    mp4_file_path
    ]
    print('\nDecoding Video information ...')
    result = subprocess.run(command_video, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    output_json = json.loads(result.stdout.decode())
    video_codec = output_json['streams'][0]['codec_name']
    pixel_format = output_json['streams'][0]['pix_fmt']
    r_frame_rate = output_json['streams'][0]['r_frame_rate']
    numerator, denominator = map(int, r_frame_rate.split('/'))
    fps = int(numerator / denominator)
    
    print(f'\nBelow are the specs for the video ', basename)
    print(f'Frames per seconds : ',fps)
    print(f'Video Codec : ',video_codec)
    print(f'Pixel Format : ',pixel_format)
    
    return fps,video_codec,pixel_format
    
    

In [6]:
def audio_info(mp4_file_path):
    command_audio = [
    'ffprobe',
    '-v', 'error',          # Hide unnecessary warnings
    '-select_streams', 'a:0', # Select the first audio stream
    '-show_entries', 'stream=codec_name', # Show the codec name
    '-of', 'json',           # Output as JSON
    mp4_file_path
    ]
    print('\nDecoding audio information ...')
    try:
        result = subprocess.run(command_audio, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        output_json = json.loads(result.stdout.decode())
        audio_codec = output_json['streams'][0]['codec_name']
        print(f'\nAudio Codec : ',audio_codec)
    except:
        print("Audio not present!")
        audio_codec=None
    
    return audio_codec

In [5]:
def audio_extract(mp4_file_path, basename):
    """
    Audio extraction from video file
    """
    video_path = mp4_file_path
    audio_path = '.mp3'

    command = [
        'ffmpeg',
        '-i', video_path, # Input video file path
        '-q:a', '0',      # Best quality audio
        '-map', 'a',      # Extract all audio streams
        audio_path        # Output audio file path
    ]   
    print('\nExtracting audio (if any)...')
    subprocess.run(command)
    return audio_path

In [7]:
def video_to_frames(mp4_file_path, output_folder_orig):
    """
    video to frames (L-R) conversion
    """
    make_folder(output_folder_orig)
    print('\nExtracting frames from Video...')
    os.system("ffmpeg -i {} ./{}/out-%05d.jpg".format(mp4_file_path,output_folder_orig))
    print(f'\nVideo frames saved in the folder ',output_folder_orig)

In [8]:
def leftOnly_frames(output_folder_orig, output_folder_left):
    """
    L-R frames to L-L frames conversion
    """
    make_folder(output_folder_left)
    images = sorted(glob.glob("./" + output_folder_orig + "/*.jpg"))
    number = 1
    print('\nExtracting L-eye image from the frames...')
    for image_path in images:
        print(image_path)
        image = Image.open(image_path)
        width, height = image.size
        left_half = (0, 0, width // 2, height)
        right_half = (width // 2, 0, width, height)
        left_image = image.crop(left_half)
        right_image = image.crop(right_half)
        new_image = Image.new("RGB", (width, height))
        new_image.paste(left_image, (0, 0))
        new_image.paste(left_image, (width // 2, 0))
        formatted_number = "{:05}".format(number)
        new_image.save("./"+output_folder_left+"/leftOnly-" + str(formatted_number) + ".jpg")
        number += 1
    print(f'\nL-eye frames saved in the folder ', output_folder_left)

In [16]:
def frames_to_videos(new_video_name, output_folder_left, fps,video_codec, audio_codec, pixel_format,audio_path):
    """
    Function to generate video from frames
    """
    frames_path = ('./'+output_folder_left+'/leftOnly-%05d.jpg') 
    output_video_name = './'+new_video_name+'.mp4'

    if audio_codec==None:
        command = [
            'ffmpeg',
            '-framerate', str(fps),
            '-i', frames_path,
            '-c:v', video_codec, 
            '-pix_fmt', pixel_format, # Pixel format
            output_video_name # Output path
        ]
    
    else:
            command = [
        'ffmpeg',
        '-framerate', str(fps),
        '-i', frames_path, # Input frames
        '-i', audio_path,  # Input audio
        '-c:v', video_codec, # Video codec
        '-c:a', audio_codec,     # Audio codec
        '-strict', 'experimental',
        '-pix_fmt', pixel_format, # Pixel format
        output_video_name # Output path
    ]
    print('\nGenerating video from L-eye frames only...')
    subprocess.run(command)
    print(f'\nL-eye only frames video saved as',output_video_name)
    

In [9]:
def filenames(mp4_file_path):
    """ 
    function to make filenames
    """
    print("Creating file names to save everything ...")
    basename = os.path.basename(mp4_file_path)  # Extract the filename: 'voyager.mp4'
    name, _ = os.path.splitext(basename)   # Split the extension: 'voyager'
    output_folder_orig = name+"_frames"   # name of the folder to save the original frames
    output_folder_left = name+"_leftFrames"   # name of the output folder for leftOnly frames
    new_video_name = name+"_lOnlyVideo"
    audio_name = name+"_audio"
    return basename, output_folder_orig, output_folder_left, new_video_name,audio_name

In [10]:
def all(mp4_file_path):     
    """ function to carry out everything"""
    basename, output_folder_orig, output_folder_left, new_video_name,audio_name = filenames(mp4_file_path)
    fps,video_codec,pixel_format = video_info(mp4_file_path, basename)
    audio_codec = audio_info(mp4_file_path)
    if(audio_codec != None):
        audio_path = audio_extract(mp4_file_path, basename)
    else:
        audio_path = None
    video_to_frames(mp4_file_path, output_folder_orig)
    leftOnly_frames(output_folder_orig, output_folder_left)
    frames_to_videos(new_video_name, output_folder_left, fps,video_codec, audio_codec, pixel_format,audio_path)
    print("Done!")

In [11]:
#Inputs from the user:
#Write the path to the .mp4 file

mp4_file_path = "./Voyager.mp4" 

#Running all functions 
all(mp4_file_path)

Creating file names to save everything ...

Decoding Video information ...

Below are the specs for the video  Voyager.mp4
Frames per seconds :  25
Video Codec :  h264
Pixel Format :  yuv420p

Decoding audio information ...
Audio not present!

Folder named Voyager_frames created! 


Extracting frames from Video...


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab